In [2]:
import os
import cv2
import numpy as np
import random

DIR_DATASET_BASE = './dataset'
DIR_VIDEOS = './mmnist-medium'

# Generación del dataset de entrenamiento

En este archivo se toma los videos de la carpeta /mmnist-medium y se le aplica el corte para que su duración sea menor y además se genera un segundo video con una mancha para tener el video de entrenamiento y su target asociado.

### Dataset de entrenamiento
![alt text](image.png)

### Dataset original
![alt text](<Captura de pantalla 2025-12-06 024702.png>)

Como se puede observar incluso con el doble de videos se obtuvo un dataset más liviano y que sigue conteniendo la información más relevante para
este proyecto. 

In [ ]:
np.random.seed(42)
random.seed(42)

def generate_circular_stain(h, w, radius=25, opacity=random.uniform(0.8, 1) * 0.7, hardness=0.8):
    # Centro aleatorio
    cx = np.random.randint(radius, w - radius)
    cy = np.random.randint(radius, h - radius)

    y, x = np.ogrid[:h, :w]
    dist = np.sqrt((x - cx)**2 + (y - cy)**2)

    # Distancia normalizada 0..1
    norm = dist / radius

    # "hardness" controla qué tan abrupto es el borde
    mask = np.clip(1 - norm, 0, 1)**hardness

    # opacidad de la mancha
    mask = mask * opacity

    return mask[..., None]

def apply_stain_to_video(frames, stain_mask):
    stained = []
    for frame in frames:
        frame_f = frame / 255.0
        stain = np.ones_like(frame_f) * stain_mask

        # Combinar (mancha oscurece + teñido)
        corrupted = frame_f * (1 - stain_mask) + (0.3 * stain)
        corrupted = np.clip(corrupted, 0, 1)
        
        stained.append((corrupted * 255).astype(np.uint8))

    return np.array(stained)


def save_video(frames, filename, fps=24):
    h, w = frames[0].shape[:2]
    writer = cv2.VideoWriter(
        filename,
        cv2.VideoWriter_fourcc(*"mp4v"),
        fps,
        (w, h)
    )
    for f in frames:
        writer.write(cv2.cvtColor(f, cv2.COLOR_RGB2BGR))
    writer.release()


In [4]:
def cortar_video(input_path, segundos):
    cap = cv2.VideoCapture(input_path)

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(fps * segundos)

    frames = []
    frame_count = 0

    while frame_count < total_frames:
        ret, frame = cap.read()
        if not ret:
            break  # fin del archivo antes del tiempo deseado
        frames.append(frame)
        frame_count += 1

    cap.release()
    
    return np.array(frames)

In [5]:
with os.scandir(DIR_VIDEOS) as elem:

    for e in elem:
        if e.is_file():
            print("Archivo:", e.name)
            video_path = os.path.join(DIR_VIDEOS, e.name)

            partes, ext = e.name.rsplit('.', 1)
            partes = partes.split('_')
            num_batch = partes[1]
            num_video = partes[3]

            os.makedirs(os.path.join(DIR_DATASET_BASE, f"batch_{num_batch}"), exist_ok=True)
            os.makedirs(os.path.join(DIR_DATASET_BASE, f"batch_{num_batch}/video_{num_video}"), exist_ok=True)

            dataset_path = f"{DIR_DATASET_BASE}/batch_{num_batch}/video_{num_video}"

            video_cortado = cortar_video(video_path, 15) # obtengo los primeros 6 segundos del video original

            mask = generate_circular_stain(video_cortado.shape[1], video_cortado.shape[2])
            video_con_manchas = apply_stain_to_video(video_cortado, mask)
            cant_manchas = random.randint(0, 2)

            for _ in range(cant_manchas):
                mask = generate_circular_stain(video_cortado.shape[1], video_cortado.shape[2])
                video_con_manchas = apply_stain_to_video(video_con_manchas, mask)


            save_video(video_cortado, f"{dataset_path}/video_original.mp4")
            save_video(video_con_manchas, f"{dataset_path}/video_con_manchas.mp4")

Archivo: batch_13_video_2.mp4
Archivo: batch_16_video_38.mp4
Archivo: batch_1_video_14.mp4
Archivo: batch_15_video_34.mp4
Archivo: batch_6_video_15.mp4
Archivo: batch_6_video_34.mp4
Archivo: batch_9_video_35.mp4
Archivo: batch_12_video_13.mp4
Archivo: batch_3_video_63.mp4
Archivo: batch_7_video_18.mp4
Archivo: batch_18_video_51.mp4
Archivo: batch_10_video_5.mp4
Archivo: batch_11_video_28.mp4
Archivo: batch_9_video_61.mp4
Archivo: batch_1_video_15.mp4
Archivo: batch_10_video_15.mp4
Archivo: batch_8_video_55.mp4
Archivo: batch_0_video_56.mp4
Archivo: batch_18_video_46.mp4
Archivo: batch_8_video_15.mp4
Archivo: batch_0_video_17.mp4
Archivo: batch_3_video_38.mp4
Archivo: batch_16_video_49.mp4
Archivo: batch_17_video_22.mp4
Archivo: batch_5_video_52.mp4
Archivo: batch_17_video_29.mp4
Archivo: batch_5_video_9.mp4
Archivo: batch_5_video_45.mp4
Archivo: batch_5_video_4.mp4
Archivo: batch_14_video_61.mp4
Archivo: batch_10_video_19.mp4
Archivo: batch_7_video_60.mp4
Archivo: batch_4_video_38.mp4
